### Manipulação de Dados com DataFrames no PySpark - Funções

#### Preparando os dados


In [0]:
spark

In [0]:
from pyspark.sql import functions as F

# Criar uma massa de dados com datas
data = [
    ("Alice", "Marketing", 5000.50, 1000, "2015-01-10", "1990-05-15"),
    ("Bob", "Marketing", 6000.75, 1500, "2017-07-22", "1988-09-12"),
    ("Charlie", "Vendas", 4000.20, 2000, "2016-11-30", "1992-12-30"),
    ("Dave", "Vendas", 4800.60, 1200, "2019-05-01", "1987-02-15"),
    ("Eve", "TI", 7000.10, 3000, "2014-03-20", "1985-08-25"),
    ("Frank", "TI", 6000.35, 2500, "2018-09-18", "1990-11-05")
]

# Definir o esquema do DataFrame
columns = ["nome", "departamento", "salario", "bonus", "data_contratacao", "data_nascimento"]

# Criar o DataFrame
df = spark.createDataFrame(data, columns)

df.createOrReplaceTempView("funcionarios")

# Mostrar o DataFrame final
display(df)



#### Funções de Manipulação de String

| Função            | Descrição                                                           | Exemplo de Entrada                  | Exemplo de Saída |
|-------------------|---------------------------------------------------------------------|-------------------------------------|------------------|
| **`upper()`**     | Converte uma string para letras maiúsculas                          | `"spark"`                           | `"SPARK"`        |
| **`lower()`**     | Converte uma string para letras minúsculas                          | `"SPARK"`                           | `"spark"`        |
| **`substring()`** | Extrai uma substring com base em uma posição inicial e comprimento  | `"Marketing"` com `substring(1, 3)` | `"Mar"`          |
| **`concat()`**    | Concatena duas ou mais strings                                      | `concat("Hello", " ", "World")`     | `"Hello World"`  |
| **`length()`**    | Retorna o comprimento de uma string                                 | `"spark"`                           | `5`              |
| **`trim()`**      | Remove os espaços em branco no início e fim de uma string           | `"  spark "`                        | `"spark"`        |
| **`lpad()`**      | Preenche a string à esquerda até atingir um determinado comprimento | `lpad("spark", 7, "x")`             | `"xxspark"`      |
| **`rpad()`**      | Preenche a string à direita até atingir um determinado comprimento  | `rpad("spark", 7, "x")`             | `"sparkxx"`      |


**Fazendo a manipulacao das Strings**

In [0]:
# Aplicar funções de manipulação de string
df = (
    df
        .withColumn("nome_maiusculo", F.upper(F.col("nome")))
        .withColumn("departamento_minusculo", F.lower(F.col("departamento")))
        .withColumn("iniciais", F.concat(F.substring("nome", 1, 1), F.substring("departamento", 1, 1)))
)
display(df)


**Equivalência em SQL**

In [0]:
%sql
SELECT 
    nome,
    UPPER(nome) AS nome_maiusculo,
    LOWER(departamento) AS departamento_minusculo,
    CONCAT(SUBSTRING(nome, 1, 1), SUBSTRING(departamento, 1, 1)) AS iniciais
FROM funcionarios;


#### Funções Matemáticas

| Função                | Descrição                                                                        | Exemplo de Entrada          | Exemplo de Saída |
|-----------------------|----------------------------------------------------------------------------------|-----------------------------|------------------|
| **`abs()`**           | Retorna o valor absoluto de um número                                            | `abs(-5000.50)`             | `5000.50`        |
| **`round()`**         | Arredonda um número para um número específico de casas decimais                  | `round(5000.567, 2)`        | `5000.57`        |
| **`ceil()`**          | Retorna o menor número inteiro maior ou igual ao valor fornecido                 | `ceil(5000.20)`             | `5001`           |
| **`floor()`**         | Retorna o maior número inteiro menor ou igual ao valor fornecido                 | `floor(5000.75)`            | `5000`           |
| **`sqrt()`**          | Retorna a raiz quadrada de um número                                             | `sqrt(5000.50)`             | `70.71774881`    |
| **`pow()`**           | Retorna um número elevado à potência de outro                                    | `pow(5000.50 + 1000, 2)`    | `36036060.25`    |
| **`format_number()`** | Formata um número com o número desejado de casas decimais e retorna como string. | `format_number(5000.50, 2)` | `"5,000.50"`     |


**Fazendo a manipulacao da coluna salario**

In [0]:
# Aplicar funções matemáticas
df = (
    df
        .withColumn("salario_abs", F.abs(F.col("salario")))
        .withColumn("salario_arredondado", F.round(F.col("salario"), 0))
        .withColumn("salario_ceil", F.ceil(F.col("salario")))
        .withColumn("salario_floor", F.floor(F.col("salario")))
        .withColumn("salario_sqrt", F.sqrt(F.col("salario")))
        .withColumn("salario_bonificado", F.pow(F.col("salario") + F.col("bonus"), 2))
        .withColumn("salario_formatado", F.format_number(F.col("salario"), 2))
)
display(df)


**Equivalência em SQL**


In [0]:
%sql
SELECT 
    nome,
    salario,
    ABS(salario) AS salario_abs,
    ROUND(salario, 2) AS salario_arredondado,
    CEIL(salario) AS salario_ceil,
    FLOOR(salario) AS salario_floor,
    SQRT(salario) AS salario_sqrt,
    POWER(salario + bonus, 2) AS salario_bonificado,
    FORMAT_NUMBER(salario, 2) AS salario_formatado
FROM funcionarios;


#### Funções de Tratamento de Datas e Horas

| Função                              | Descrição                                                                                  | Exemplo de Entrada                                                | Exemplo de Saída        |
|-------------------------------------|--------------------------------------------------------------------------------------------|-------------------------------------------------------------------|-------------------------|
| **`current_date()`**                | Retorna a data atual.                                                                      | `current_date()`                                                  | `2024-09-24`            |
| **`datediff(end, start)`**          | Calcula a diferença de dias entre duas datas.                                              | `datediff('2024-09-24', '2015-01-10')`                            | `3520`                  |
| **`floor()`**                       | Arredonda o número para o valor inteiro mais próximo, útil para transformar dias em anos.  | `floor(3520 / 365)`                                               | `9` (anos de diferença) |
| **`add_months(date, n)`**           | Adiciona ou subtrai um número de meses a uma data.                                         | `add_months('1990-05-15', 12)`                                    | `1991-05-15`            |
| **`year()`**                        | Extrai o ano de uma data.                                                                  | `year('2024-09-24')`                                              | `2024`                  |
| **`month()`**                       | Extrai o mês de uma data.                                                                  | `month('2024-09-24')`                                             | `9`                     |
| **`dayofmonth()`**                  | Extrai o dia do mês de uma data.                                                           | `dayofmonth('2024-09-24')`                                        | `24`                    |
| **`date_format()`**                 | Formata uma data em um padrão específico.                                                  | `date_format('2024-09-24', 'dd/MM/yyyy')`                         | `24/09/2024`            |
| **`next_day(date, 'day_of_week')`** | Retorna a data do próximo dia da semana a partir de uma data fornecida.                    | `next_day('2024-09-24', 'Mon')`                                   | `2024-09-30`            |
| **`to_date()`**                     | Converte uma string que representa uma data em uma coluna de data no formato especificado. | `to_date('15-05-1990', 'dd-MM-yyyy')`                             | `1990-05-15`            |
| **`to_timestamp()`**                | Converte uma string com data e hora em um timestamp (com fuso horário).                    | `to_timestamp('2024-09-24 13:45:00', 'yyyy-MM-dd HH:mm:ss')`      | `2024-09-24 13:45:00`   |
| **`to_timestamp_ntz()`**            | Converte uma string para um timestamp sem fuso horário.                                    | `to_timestamp_ntz('2024-09-24 13:45:00', 'yyyy-MM-dd HH:mm:ss')`  | `2024-09-24 13:45:00`   |
| **`to_timestamp_ltz()`**            | Converte uma string para um timestamp com fuso horário local (local time zone).            | `to_timestamp_ltz('2024-09-24 13:45:00', 'yyyy-MM-dd HH:mm:ss')`  | `2024-09-24 13:45:00`   |
| **`to_unix_timestamp()`**           | Converte uma string com data e hora em um timestamp Unix (segundos desde 01/01/1970 UTC).  | `to_unix_timestamp('2024-09-24 13:45:00', 'yyyy-MM-dd HH:mm:ss')` | `1727174700`            |
| **`to_utc_timestamp()`**            | Converte um timestamp local para UTC (tempo universal).                                    | `to_utc_timestamp('2024-09-24 13:45:00', 'PST')`                  | `2024-09-24 21:45:00`   |
| **`date_add(date, num_days)`**      | Adiciona ou subtrai um número de dias a partir de uma data.                                | `date_add('2024-09-24', 10)`                                      | `2024-10-04`            |



**Fazendo a manipulacao das colunas data_contratacao e data_nascimento**

In [0]:
# Converter colunas de string para data
df = (
    df
    .withColumn("data_contratacao", F.to_date(F.col("data_contratacao"), "yyyy-MM-dd"))
    .withColumn("data_nascimento", F.to_date(F.col("data_nascimento"), "yyyy-MM-dd"))
)
display(df)

In [0]:
# Funções de tratamento de datas
df = (
    df
        .withColumn("idade", F.floor(F.datediff(F.current_date(), F.col("data_nascimento")) / 365)) 
        .withColumn("tempo_empresa", F.floor(F.datediff(F.current_date(), F.col("data_contratacao")) / 365))
        .withColumn("proximo_aniversario", F.add_months(F.col("data_nascimento"), 12 * (F.year(F.current_date()) - F.year(F.col("data_nascimento")))))
        .withColumn("ano_contratacao", F.year(F.col("data_contratacao"))) 
        .withColumn("mes_contratacao", F.month(F.col("data_contratacao"))) 
        .withColumn("dia_contratacao", F.dayofmonth(F.col("data_contratacao")))
)
display(df)


**Equivalência em SQL**


In [0]:
%sql
SELECT 
    nome,
    departamento,
    salario,
    bonus,
    data_contratacao,
    data_nascimento,
    FLOOR(DATEDIFF(CURRENT_DATE(), data_nascimento) / 365) AS idade,
    FLOOR(DATEDIFF(CURRENT_DATE(), data_contratacao) / 365) AS tempo_empresa,
    ADD_MONTHS(data_nascimento, 12 * (YEAR(CURRENT_DATE()) - YEAR(data_nascimento))) AS proximo_aniversario,
    YEAR(data_contratacao) AS ano_contratacao,
    MONTH(data_contratacao) AS mes_contratacao,
    DAYOFMONTH(data_contratacao) AS dia_contratacao
FROM funcionarios;

`[INFO]: FIM DO NOTEBOOK`